<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projeto Proposto 
</h1>
<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este projeto tem o intuito de estudar Programação Orientada a Objetos pois ela é a estrutura de códigos de Deep Learning </i>       
</p>  

# <font color='red' style='font-size: 40px;'> Bibliotecas Utilizadas </font>
<hr style='border: 2px solid red;'>

In [1]:
## Bibliotecas De Manipulação de Dados e Visualização
import pandas as pd 
import geopandas as gpd
import builtins as builtins
import matplotlib.pyplot as plt
import seaborn as sns 
import itertools
from IPython.display import display, Image

## Bibliotecas de Modelagem Matemática e Estatística
import numpy as np
import scipy as sp 
import scipy.stats as stats
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA


# Bibliotecas de Manipulação de Tempo
import time
import datetime

# Bibliotecas de Métricas de Machine Learning
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Parâmetros de Otimização
import warnings
%matplotlib inline
sns.set()
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# <font color='red' style='font-size: 40px;'> Funções </font>
<hr style='border: 2px solid red;'>

# <font color='green' style='font-size: 30px;'> 1.1) Tema 1 </font>
<hr style='border: 2px solid green;'>

In [ ]:
# nn.Linear(a, b) cria pesos e bias para uma transformação linear x @ W^T + b.
# F.relu(...) aplica a função ReLU elemento-a-elemento (max(0, x)).
# hidden_dims é um argumento com valor padrão; cuidado com mutáveis como listas em assinaturas de função (não é crítico aqui, mas é um ponto de atenção em geral).

class MLP_PyTorch(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32]):
        super(MLP_PyTorch, self).__init__()               # Chama o construtor da classe base (nn.Module).
        self.fc1 = nn.Linear(input_dim, hidden_dims[0])   # Primeira camada linear: transforma input_dim -> hidden_dims[0].
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])  # Segunda camada linear: hidden_dims[0] -> hidden_dims[1].
        self.out = nn.Linear(hidden_dims[1], 1)           # Camada de saída: hidden_dims[1] -> 1 (probabilidade do label 1).
    
    def forward(self, x):
        x = F.relu(self.fc1(x))                           # Aplica fc1 e ReLU (não linearidade). Saída shape: (batch, hidden_dims[0])
        x = F.relu(self.fc2(x))                           # Aplica fc2 e ReLU. Saída shape: (batch, hidden_dims[1])
        x = torch.sigmoid(self.out(x))                    # Aplica a camada final e sigmoid -> probabilidade entre 0 e 1.
        return x                                         # Retorna tensor de probabilidades shape (batch, 1)
    
################ CUDA ##################
# CUDA (Compute Unified Device Architecture) é a plataforma da NVIDIA que permite usar a placa de vídeo (GPU) não só para gráficos, mas também para cálculos científicos e de machine learning.
# O PyTorch pode rodar tanto em CPU quanto em GPU.
# Quando você instala o PyTorch com CUDA habilitado, ele sabe "conversar" com sua GPU NVIDIA para mandar os tensores e operações para lá.
# Isso acelera absurdamente o treinamento de redes neurais — em vez de horas na CPU, você pode treinar em minutos na GPU.
# Rodar no PROMPT nvidia-smi para saber a versão da sua GPU
# conda create -n pytorch_cuda python=3.9
# conda activate pytorch_cuda
# conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

def train_mlp_pytorch(
        X_train, 
        y_train, 
        class_weight, 
        lr=0.001, # Learning Rate
        epochs=50, 
        batch_size=256, 
        device='cuda' # "cuda" é GPU e "cpu" é CPU
    ):
    # Converte os dados de treino (features) para tensores float32 e envia para a GPU/CPU
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)

    # Converte o target (labels) para tensores float32, adiciona dimensão extra (coluna) e envia para GPU/CPU
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).to(device)
    
    # Cria um dataset do PyTorch unindo features e labels
    dataset = TensorDataset(X_train_tensor, y_train_tensor)

    # Cria um DataLoader que divide o dataset em batches e embaralha os dados
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Instancia o modelo MLP definido anteriormente e envia para GPU/CPU
    model = MLP_PyTorch(input_dim=X_train.shape[1]).to(device)
    
    # Cria um tensor de pesos para lidar com desbalanceamento de classes (class_weight)
    weight_tensor = torch.tensor(
        [class_weight if label==1 else 1.0 for label in y_train.values],  # aplica peso maior para a classe 1
        dtype=torch.float32
    ).unsqueeze(1).to(device)

    # Define a função de perda Binary Cross-Entropy (BCELoss) com pesos por amostra
    criterion = nn.BCELoss(weight=weight_tensor)
    
    # Define o otimizador Adam com taxa de aprendizado lr
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Coloca o modelo em modo de treino
    model.train()

    # Loop principal de treino
    for epoch in range(epochs):  # repete pelo número de épocas
        for xb, yb in loader:    # itera sobre os batches do DataLoader
            optimizer.zero_grad()       # zera os gradientes acumulados
            preds = model(xb)           # passa o batch pelo modelo (forward)
            loss = criterion(preds, yb) # calcula a perda (comparando preds vs rótulos reais)
            loss.backward()             # faz backpropagation (calcula gradientes)
            optimizer.step()            # atualiza os pesos do modelo
    
    # Retorna o modelo treinado
    return model

def predict_mlp_pytorch(model, X, device='cuda'):
    # Coloca o modelo em modo de avaliação (desliga dropout e batchnorm em modo treino)
    model.eval()
    
    # Converte o DataFrame X em tensor float32 e envia para o dispositivo especificado (GPU ou CPU)
    X_tensor = torch.tensor(X.values, dtype=torch.float32).to(device)
    
    # Bloco que desativa o cálculo de gradientes para economizar memória e acelerar
    with torch.no_grad():
        # Passa os dados pelo modelo e obtém as probabilidades preditas
        # .cpu() garante que o resultado volte para a CPU
        # .numpy() converte o tensor em array NumPy
        probs = model(X_tensor).cpu().numpy()
    
    # Converte as probabilidades em rótulos binários: 1 se >= 0.5, 0 se < 0.5
    preds = (probs >= 0.5).astype(int)
    
    # Retorna os rótulos preditos e as probabilidades
    return preds, probs

